In [141]:
#!pip install selenium

In [142]:
from PIL import Image
import io
import requests
import hashlib
import os
import time

In [143]:
import selenium
from selenium import webdriver
# Put the path for your ChromeDriver here
DRIVER_PATH = r"C:/webdriver/chromedriver.exe" 



In [144]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=5):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img[alt*='Lettuce']")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img[alt*='Lettuce']')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector("img[alt*='Lettuce']")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [145]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [146]:
#You can change number of images from here
def search_and_download(search_term:str,driver_path:str,target_path=r"C:/Users/Reddy/Desktop/selenium",number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=5)
        
    for elem in res:
        persist_image(target_folder,elem)

### Just run this cell by putting your search keyword

In [147]:
search_term = 'gray mould lettuce disease'
search_and_download(search_term = search_term,driver_path= DRIVER_PATH)

Found: 21 search results. Extracting links from 0:21
Found: 109 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvAfuk4_aoAo_8XkjC62fvd5rnhoNGYtIW6w&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\5e16eb2003.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQa3eiSPLGTlCMxp9ePNK-WoXDesO56KoXnig&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\9e1282ca3b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRSA85omeZq8xjq6NZVP1UF4IBns9xGx9vFJQ&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\85e8adbdb5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQt9ctVGMmLKA10Vz47pCLqui2aHs-O_c0k6w&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\e7e24d2bd3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSRzBkROGJN_4VysA_qplFJa3D41_nEAMLbZg&usqp=CAU - as C:/Users/Reddy/D

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSv295DQ0uvL0I1LH6SLMrPHdS26iioKBqumw&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\64fd06d7f1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS-yZB-CvibiK7klJ5wc4n9DIoedQu5fd9PLA&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\f9cce8776a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQG2Xo7TppL_NXPYpuQMr3hK4AVqzck798XAA&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\3311a6bbda.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQS4hZvXCC2O65QLZaLuZEpLW4QWBYN6a8dMw&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\0792f132b1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ5pFMWsvcoA6xI5MwP4Z2uu59gZs_OMo0dTw&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\92e33946bd.jpg
SUCCESS - saved https://e

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQhzBvQZotcIMgGtnjszQeuwD4ReedKXskhpA&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\9a373d61a8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ9wDct_AffmdNbKAPgk4j_u7afrg0qqCnyXw&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\35e69764de.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ685sPd-A4cBxnvrdgKjx0br1FS41kSWadZw&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\c7138b72f2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2oyo96a0Up9D0fjHoiutGJcWz2pyTVNH52Q&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\3655e6ee91.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRAv3l_DGg-XT8rbFrj86mdmmKY9wtkIh8ckw&usqp=CAU - as C:/Users/Reddy/Desktop/selenium\gray_mould_lettuce_disease\4b53b66e3c.jpg
SUCCESS - saved https://e